In [1]:
# Install downgraded version of numpy and restart the runtime otherwise you will get error
# "ValueError: Object arrays cannot be loaded when allow_pickle=False".
# This issue is still up on keras git. I hope it gets solved as soon as possible
!pip install numpy==1.16.1

  Using cached numpy-1.16.1.zip (5.1 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [2221 lines of output]
      Running from numpy source directory.
      /private/var/folders/9d/7f20ppnx1tndyhj_ryqldllm0000gn/T/pip-install-p7ybn_ff/numpy_91d1a10b3bc24d4188f525715806ffd4/numpy/distutils/misc_util.py:476: SyntaxWarning: "is" with a literal. Did you mean "=="?
        return is_string(s) and ('*' in s or '?' is s)
      blas_opt_info:
      blas_mkl_info:
      customize UnixCCompiler
        FOUND:
          libraries = ['mkl_rt', 'pthread']
          library_dirs = ['/Users/kivsithvothy/opt/anaconda3/lib']
          define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
          include_dirs = ['/usr/local/include', '/Users/kivsithvothy/opt/anaconda3/include']
      
        FOUND:
          libraries = ['mkl_rt', 'pthread']
          library_dirs = ['/Users/

In [ ]:
!pip install keras
!pip install tensorflow

In [3]:
#Import Libraries


# from keras.layers import LSTM, Dropout
# from keras.layers import Embedding
# from keras.models import load_model
# -------------------------------------------------------------

import numpy
from numpy import array
from keras.datasets import imdb
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Dropout, Embedding

import re
import numpy as np
from nltk.tokenize import word_tokenize
import nltk

from keras.utils import pad_sequences # in tf version 2.9 it is under keras.utils
# from keras.preprocessing import sequence

# fix random seed for reproducibility
numpy.random.seed(7)

2024-03-01 10:05:44.694745: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/kivsithvothy/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# This is a dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative).
# Reviews have been preprocessed, and each review is encoded as a list of word indexes (integers).

# Load the dataset but only keep the top n words and zero out the rest i.e keep vocabulary size as 5000
top_words = 5000 # vocabulary_size = 5000 num_words=top_words
(X_train, y_train), (X_test, y_test) = imdb.load_data()

In [ ]:
(X_train, y_train), (X_test, y_test)

In [6]:
# Inspect a sample review and its label.Note that the review is stored as a sequence of integers. These are word IDs that
# have been pre-assigned to individual words, and the label is an integer (0 for negative, 1 for positive).

print('---review---')
print(X_train[6])
print('---label---')
print(y_train[6])


---review---
[1, 6740, 365, 1234, 5, 1156, 354, 11, 14, 5327, 6638, 7, 1016, 10626, 5940, 356, 44, 4, 1349, 500, 746, 5, 200, 4, 4132, 11, 16393, 9363, 1117, 1831, 7485, 5, 4831, 26, 6, 71690, 4183, 17, 369, 37, 215, 1345, 143, 32677, 5, 1838, 8, 1974, 15, 36, 119, 257, 85, 52, 486, 9, 6, 26441, 8564, 63, 271, 6, 196, 96, 949, 4121, 4, 74170, 7, 4, 2212, 2436, 819, 63, 47, 77, 7175, 180, 6, 227, 11, 94, 2494, 33740, 13, 423, 4, 168, 7, 4, 22, 5, 89, 665, 71, 270, 56, 5, 13, 197, 12, 161, 5390, 99, 76, 23, 77842, 7, 419, 665, 40, 91, 85, 108, 7, 4, 2084, 5, 4773, 81, 55, 52, 1901]
---label---
1


In [7]:
# We can use the dictionary returned by imdb.get_word_index() to map the review back to the original words.

word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}

print('---review with words---')
print([id2word.get(i, ' ') for i in X_train[6]])
print('---label---')


---review with words---
['the', 'boiled', 'full', 'involving', 'to', 'impressive', 'boring', 'this', 'as', 'murdering', 'naschy', 'br', 'villain', 'council', 'suggestion', 'need', 'has', 'of', 'costumes', 'b', 'message', 'to', 'may', 'of', 'props', 'this', 'echoed', 'concentrates', 'concept', 'issue', 'skeptical', 'to', "god's", 'he', 'is', 'dedications', 'unfolds', 'movie', 'women', 'like', "isn't", 'surely', "i'm", 'rocketed', 'to', 'toward', 'in', "here's", 'for', 'from', 'did', 'having', 'because', 'very', 'quality', 'it', 'is', "captain's", 'starship', 'really', 'book', 'is', 'both', 'too', 'worked', 'carl', 'of', 'mayfair', 'br', 'of', 'reviewer', 'closer', 'figure', 'really', 'there', 'will', 'originals', 'things', 'is', 'far', 'this', 'make', 'mistakes', "kevin's", 'was', "couldn't", 'of', 'few', 'br', 'of', 'you', 'to', "don't", 'female', 'than', 'place', 'she', 'to', 'was', 'between', 'that', 'nothing', 'dose', 'movies', 'get', 'are', '498', 'br', 'yes', 'female', 'just', 'it

In [ ]:
word2id

In [ ]:
print(y_train[6])

In [ ]:
word2id

In [ ]:
id2word

In [9]:
# Maximum review length and minimum review length.
print('Maximum review length: {}'.format(len(max((X_train + X_test), key=len))))


Maximum review length: 2697


In [10]:
print('Minimum review length: {}'.format(len(min((X_train + X_test), key=len))))


Minimum review length: 70


In [11]:

# In order to feed this data into our RNN, all input documents must have the same length.
# We will limit the maximum review length to maximum words by truncating
# longer reviews and padding shorter reviews with a null value (0).
# We can accomplish this task using the pad_sequences() function in Keras.
# Here, setting max_review_length to 1000 or 500.

max_review_length = 1000 #500
X_train = pad_sequences(X_train, maxlen=max_review_length)
X_test = pad_sequences(X_test, maxlen=max_review_length)


## Train the model

In [12]:

# Remember that our input is a sequence of words (technically, integer word IDs) of maximum length = max_review_length,
# and our output is a binary sentiment label (0 or 1).


# Create model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# We first need to compile our model by specifying the loss function and
# optimizer we want to use while training, as well as any evaluation metrics we’d like to measure.
# Specify the appropriate parameters, including at least one metric ‘accuracy’.


# Train
# -----------------------------------------------------------------------------------
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=64)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 32)          160000    
                                                                 
 dropout (Dropout)           (None, 1000, 32)          0         
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 213301 (833.21 KB)
Trainable params: 213301 (833.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/30


InvalidArgumentError: Graph execution error:

Detected at node sequential/embedding/embedding_lookup defined at (most recent call last):
  File "/Users/kivsithvothy/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/Users/kivsithvothy/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code

  File "/Users/kivsithvothy/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>

  File "/Users/kivsithvothy/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance

  File "/Users/kivsithvothy/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start

  File "/Users/kivsithvothy/opt/anaconda3/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start

  File "/Users/kivsithvothy/opt/anaconda3/lib/python3.9/asyncio/base_events.py", line 601, in run_forever

  File "/Users/kivsithvothy/opt/anaconda3/lib/python3.9/asyncio/base_events.py",

## Calculate Accuracy


In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
#Run these codes first in order to install the necessary libraries and perform authorization
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


## Mount your Google Drive:

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
#After success run Drive FUSE program, you can create a directory Sentiment_Analysis and access your drive at /content/drive with using command
import os
os.mkdir("/content/drive/Sentiment_Analysis")
os.chdir("/content/drive/")
!ls

In [ ]:
#Append your path
import sys
sys.path.append('/content/drive/Sentiment_Analysis')

## Save the model

In [ ]:
#Now save the model in required directory
model.save('/content/drive/Sentiment_Analysis/sentiment_analysis_model_new.h5')
print("Saved model to disk")

In [ ]:
#Check the content of the directory
os.chdir("/content/drive/Sentiment_Analysis")
!ls

In [ ]:
#Code to load the saved model
model = load_model('/content/drive/Sentiment_Analysis/sentiment_analysis_model_new.h5')
print("Model Loaded")